# Sora2 Watermark Remover (Google Colab版)

このノートブックは、Sora動画からウォーターマークを除去するツールのGoogle Colab版です。

**新機能:**
- `--frame-step`: フレーム処理間隔（1=全フレーム、2=隔フレーム...）
- `--target-fps`: 出力fps指定（0=入力と同じ）

**リポジトリ:** https://github.com/fulfulggg/Sora2WatermarkRemover

## 1. セットアップ

### GitHubからリポジトリをクローン

In [ ]:
# GitHubリポジトリのクローン
!git clone https://github.com/fulfulggg/Sora2WatermarkRemover.git
%cd Sora2WatermarkRemover

# 取得＆ブランチ切替（main優先、無ければfeat/frame-step）
!git fetch origin --prune
!git checkout main || git checkout feat/frame-step
!git pull --ff-only

print("\n✅ リポジトリの準備が完了しました（main優先）。")

### ⚠️ 重要: Pillow依存関係の修正

**以下のセルを実行し、その後必ずランタイムを再起動してください。**

1. 下のセルを実行
2. 「ランタイム」→「ランタイムを再起動」を選択
3. 再起動後、次のセルから実行を続ける

In [ ]:
# Pillow依存関係の修正（実行後にランタイムを再起動してください）
!pip uninstall -y pillow PIL
!pip install -U "pillow==10.4.0"

# オプション: websockets警告の抑制
!pip install -U "websockets==15.0.1"

print("\n✅ 完了しました。ランタイム→ランタイムを再起動 を実行してください。")

In [ ]:
# 再起動後の確認（このセルを最初に実行）
import PIL
import PIL._util as u

print(f"Pillow version: {PIL.__version__}")  # => 10.4.0
print(f"is_directory exists: {hasattr(u, 'is_directory')}")  # => True

if PIL.__version__ == "10.4.0" and hasattr(u, "is_directory"):
    print("✅ Pillow依存関係は正常です。次のセルに進んでください。")
else:
    print("⚠️ 問題があります。ランタイムを再起動しましたか？")

In [ ]:
# 必要なパッケージのインストール
# 0) 既存の競合パッケージをアンインストール（環境整合のため）
!pip uninstall -y diffusers huggingface_hub transformers

# 1) 基本ライブラリ + iopaint
!pip install -q transformers torch opencv-python tqdm loguru iopaint

# 2) 互換性ピン（iopaint と transformers の両立）
!pip install -q "diffusers==0.26.3" "huggingface_hub>=0.34,<1.0" "transformers>=4.45,<5"

!apt-get -qq install -y ffmpeg

print("✅ パッケージのインストールが完了しました（iopaint/transformers 互換ピン適用）。")

# バージョン確認（任意）
import sitecustomize
import huggingface_hub, diffusers, transformers
print(f"huggingface_hub: {huggingface_hub.__version__}")
print(f"diffusers      : {diffusers.__version__}")
print(f"transformers   : {transformers.__version__}")

## 2. デバイス確認

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPUが利用できません。ランタイム→ランタイムのタイプを変更→GPUを選択してください。")

## 3. 動画アップロード

In [ ]:
from google.colab import files

print("動画ファイルをアップロードしてください:")
uploaded = files.upload()

# アップロードされたファイルを取得
input_video = list(uploaded.keys())[0]
print(f"\nUploaded: {input_video}")

## 4. 動画処理

### パラメータ設定

- `max_bbox_percent`: バウンディングボックスの最大サイズ（画像に対する割合%）
- `frame_step`: フレーム処理間隔（1=全フレーム、2=隔フレーム...）
- `target_fps`: 出力fps（0=入力と同じ）

In [ ]:
# パラメータ設定（必要に応じて調整）
max_bbox_percent = 10.0  # バウンディングボックスの最大サイズ
frame_step = 1           # 1=全フレーム、2=隔フレーム
target_fps = 0.0         # 0=入力と同じ

output_video = "output_no_watermark.mp4"

# ディレクトリ移動
%cd /content/Sora2WatermarkRemover

# 入力ファイルの確認
import os
input_path = f"/content/{input_video}"
output_path = f"/content/{output_video}"

if not os.path.exists(input_path):
    print(f"❌ エラー: 入力ファイルが見つかりません: {input_path}")
else:
    print(f"✅ 入力ファイル確認: {input_path}")
    print(f"📝 出力先: {output_path}")
    print(f"⚙️ パラメータ: max_bbox_percent={max_bbox_percent}, frame_step={frame_step}, target_fps={target_fps}")
    print("\n🚀 処理を開始します...\n")
    
    # remwm.pyを実行（所要時間を計測）
    import time
    t0 = time.time()
    !python remwm.py "{input_path}" "{output_path}" \
      --max-bbox-percent {max_bbox_percent} \
      --frame-step {frame_step} \
      --target-fps {target_fps}
    print(f"⏱ Elapsed: {time.time() - t0:.1f}s")
    
    # 出力ファイルの確認
    if os.path.exists(output_path):
        file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB
        print(f"\n✅ 処理完了: {output_path}")
        print(f"📊 ファイルサイズ: {file_size:.2f} MB")
    else:
        print(f"\n❌ エラー: 出力ファイルが作成されませんでした")
        print("上記のエラーメッセージを確認してください")

## 5. 結果のダウンロード

In [ ]:
# 結果のダウンロード
print("結果をダウンロード:")
files.download(f"/content/{output_video}")

## 6. 使い方のヒント

### 処理速度を上げたい場合

```python
frame_step = 2  # 隔フレーム処理（約2倍速）
```

### 出力fpsを固定したい場合

```python
target_fps = 30  # 30fpsで出力
```

### 組み合わせ例

```python
frame_step = 2
target_fps = 30
# → 隔フレーム処理して30fpsで出力
```